In [1]:
%load_ext autoreload
%autoreload 2 
%reload_ext autoreload
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
import scipy.io as io
from pyDOE import lhs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

from complexPyTorch.complexLayers import ComplexLinear

import cplxmodule
from cplxmodule import cplx
from cplxmodule.nn import RealToCplx, CplxToReal, CplxSequential, CplxToCplx
from cplxmodule.nn import CplxLinear, CplxModReLU, CplxAdaptiveModReLU, CplxModulus, CplxAngle

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from lightning_utils import *
from utils import *
from models import (TorchComplexMLP, ImaginaryDimensionAdder, cplx2tensor, 
                    ComplexTorchMLP, ComplexSymPyModule, complex_mse)
from preprocess import *

# Model selection
from sparsereg.model import STRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from RegscorePy.bic import bic

from madgrad import MADGRAD

/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# torch device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You're running on", device)

# Doman bounds
lb = np.array([-5.0, 0.0])
ub = np.array([5.0, np.pi/2])

DATA_PATH = '../experimental_data/NLS.mat'
data = io.loadmat(DATA_PATH)

t = data['tt'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = data['uu']
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = to_column_vector(Exact_u.T)
v_star = to_column_vector(Exact_v.T)

N = 500
idx = np.random.choice(X_star.shape[0], N, replace=False)
# idx = np.arange(N) # Just have an easy dataset for experimenting

lb = to_tensor(lb, False).to(device)
ub = to_tensor(ub, False).to(device)

X_train = to_tensor(X_star[idx, :], True).to(device)
u_train = to_tensor(u_star[idx, :], False).to(device)
v_train = to_tensor(v_star[idx, :], False).to(device)

feature_names = ['hf', '|hf|', 'h_xx']

### Loading data code here ###
print("Loading pre-calculated data for reproducibility")
X_train = to_tensor(np.load("./tmp_files/X_train_500+500samples.npy"), True)[:N, :]
u_train, v_train = dimension_slicing(to_tensor(np.load("./tmp_files/uv_train_500samples.npy"), False))
u_train = u_train[:N, :]
v_train = v_train[:N, :]
h_train = torch.complex(u_train, v_train)
### ----- ###

You're running on cpu
Loading pre-calculated data for reproducibility


In [3]:
cn1 = 0.002494+1.002397*1j
cn2 = 0.003655+0.500415*1j
cns = [cn1, cn2]

In [4]:
# Type the equation got from the symbolic regression step
# No need to save the eq save a pickle file before
program1 = "X0*X1"
pde_expr1, variables1,  = build_exp(program1); print(pde_expr1, variables1)

program2 = "X2"
pde_expr2, variables2,  = build_exp(program2); print(pde_expr2, variables2)

mod = ComplexSymPyModule(expressions=[pde_expr1, pde_expr2], complex_coeffs=cns); mod.train()

Please ensure that all coefs are different
X0*X1 {X1, X0}
Please ensure that all coefs are different
X2 {X2}


ComplexSymPyModule(
  (sympymodule): SymPyModule(expressions=(X0*X1, X2))
)

In [5]:
class ComplexPINN(nn.Module):
    def __init__(self, model, loss_fn, index2features, scale=False, lb=None, ub=None):
        super(ComplexPINN, self).__init__()
        self.model = model
        self.callable_loss_fn = loss_fn
        self.index2features = index2features; self.feature2index = {}
        for idx, fn in enumerate(self.index2features): self.feature2index[fn] = str(idx)
        self.scale = scale; self.lb, self.ub = lb, ub
        if self.scale and (self.lb is None or self.ub is None): 
            print("Please provide thw lower and upper bounds of your PDE.")
            print("Otherwise, there will be error(s)")
        self.diff_flag = diff_flag(self.index2features)
        
    def forward(self, x, t):
        H = torch.cat([x, t], dim=1)
        if self.scale: H = self.neural_net_scale(H)
        return self.model(H)
    
    def loss(self, x, t, y_input, update_network_params=True, update_pde_params=True):
        total_loss = []
        grads_dict, u_t = self.grads_dict(x, t)
        # MSE Loss
        if update_network_params:
            mse_loss = complex_mse(grads_dict['X'+self.feature2index['hf']], y_input)
            total_loss.append(mse_loss)
        # PDE Loss
        if update_pde_params:
            l_eq = complex_mse(self.callable_loss_fn(grads_dict), u_t)
            total_loss.append(l_eq)
            
        return total_loss
    
    def grads_dict(self, x, t):
        uf = self.forward(x, t)
        u_t = complex_diff(uf, t)
        
        ### PDE Loss calculation ###
        # Without calling grad
        derivatives = {}
        for t in self.diff_flag[0]:
            if t=='hf': 
                derivatives['X'+self.feature2index[t]] = cplx2tensor(uf)
                derivatives['X1'] = (uf.real**2+uf.imag**2)+0.0j
            elif t=='x': derivatives['X'+self.feature2index[t]] = x
        # With calling grad
        for t in self.diff_flag[1]:
            out = uf
            for c in t:
                if c=='x': out = complex_diff(out, x)
                elif c=='t': out = complex_diff(out, t)
            derivatives['X'+self.feature2index['h_'+t[::-1]]] = out
        
        return derivatives, u_t
    
    def gradients(self, func, x):
        return grad(func, x, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape))
    
    # Must ensure that the implementation of neural_net_scale is consistent
    # and hopefully correct
    # also, you might not need this function in some datasets
    def neural_net_scale(self, inp): 
        return 2*(inp-self.lb)/(self.ub-self.lb)-1

In [6]:
inp_dimension = 2
act = CplxToCplx[torch.tanh]
complex_model = CplxSequential(
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 1, bias=True),
                            )
complex_model = torch.nn.Sequential(
                                    torch.nn.Linear(inp_dimension, 200),
                                    RealToCplx(),
                                    complex_model
                                    )

/usr/local/lib/python3.9/site-packages/torch/nn/modules/container.py:587: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")


In [7]:
# Pretrained model
semisup_model_state_dict = cpu_load("./saved_path_inverse_nls/NLS_complex_model_500labeledsamples_jointtrainwith500unlabeledsamples.pth")
parameters = OrderedDict()

# Filter only the parts that I care about renaming (to be similar to what defined in TorchMLP).
inner_part = "network.model."
for p in semisup_model_state_dict:
    if inner_part in p:
        parameters[p.replace(inner_part, "")] = semisup_model_state_dict[p]
complex_model.load_state_dict(parameters)

pinn = ComplexPINN(model=complex_model, loss_fn=mod, index2features=feature_names, scale=False, lb=lb, ub=ub)

In [8]:
def closure():
    global X_train, h_train
    if torch.is_grad_enabled():
        optimizer2.zero_grad()
    losses = pinn.loss(X_train[:, 0:1], X_train[:, 1:2], h_train, update_network_params=True, update_pde_params=True)
    l = sum(losses)
    if l.requires_grad:
        l.backward(retain_graph=True)
    return l

def mtl_closure():
    global X_train, h_train
    n_obj = 2 # There are two tasks
    losses = pinn.loss(X_train[:, 0:1], X_train[:, 1:2], h_train, update_network_params=True, update_pde_params=True)
    updated_grads = []
    
    for i in range(n_obj):
        optimizer1.zero_grad()
        losses[i].backward(retain_graph=True)

        g_task = []
        for param in pinn.parameters():
            if param.grad is not None:
                g_task.append(Variable(param.grad.clone(), requires_grad=False))
            else:
                g_task.append(Variable(torch.zeros(param.shape), requires_grad=False))
        # appending the gradients from each task
        updated_grads.append(g_task)

    updated_grads = list(pcgrad.pc_grad_update(updated_grads))[0]
    for idx, param in enumerate(pinn.parameters()): 
        param.grad = (updated_grads[0][idx]+updated_grads[1][idx])
        
    return sum(losses)

In [9]:
# epochs1, epochs2 = 200, 50
# # TODO: Save best state dict and training for more epochs.
# optimizer1 = MADGRAD(pinn.parameters(), lr=1e-7, momentum=0.9)
# pinn.train(); best_train_loss = 1e6

# print('1st Phase optimization using Adam with PCGrad gradient modification')
# for i in range(epochs1):
#     optimizer1.step(mtl_closure)
#     l = mtl_closure()
#     if (i % 10) == 0 or i == epochs1-1:
#         print("Epoch {}: ".format(i), l.item())

In [10]:
# optimizer2 = torch.optim.LBFGS(pinn.parameters(), lr=1e-1, max_iter=300, max_eval=int(300*1.25), history_size=150, line_search_fn='strong_wolfe')
# print('2nd Phase optimization using LBFGS')
# for i in range(epochs2):
#     optimizer2.step(closure)
#     l = closure()
#     if (i % 5) == 0 or i == epochs2-1:
#         print("Epoch {}: ".format(i), l.item())

In [11]:
# torch.save(pinn.state_dict(), "./saved_path_inverse_nls/final_finetuned_cpinn.pth")

In [12]:
# est_coeffs = pinn.callable_loss_fn.complex_coeffs().detach().numpy().ravel()
# grounds = np.array([1j, 0+0.5j])

# errs = []
# for i in range(len(grounds)):
#     err = est_coeffs[i]-grounds[i]
#     errs.append(100*abs(err.imag)/abs(grounds[i].imag))
# errs = np.array(errs)
# errs.mean(), errs.std()

#### Fun with lightning for simple best-practice finetuning procedure

In [13]:
# Be aware of the double neural net scaling
class LightningComplexPINN(ParentFinetuner):
    # Parent's args + additional args
    def __init__(self, model, inp_scale=False, bounds=None, max_epochs=1000, lr=1e-3, n_obj=2):
        super(LightningComplexPINN, self).__init__(model, inp_scale, bounds, max_epochs=1000)
        self.n_obj = n_obj
        self.lr = lr
        
    def forward(self, *args):
        return self.model(*args)
    
    def configure_optimizers(self):
        return MADGRAD(self.parameters(), lr=self.lr, momentum=0.9)
        
    def training_step(self, train_batch, batch_idx):
        myopt = self.optimizers()
        x, y = train_batch; x = x.view(x.size(0), -1)
        spatial, time = dimension_slicing(x)
        losses = self.model.loss(spatial, time, y, update_network_params=True, update_pde_params=True)
        
        # Before calling the trainer.fit function
        # automatic optimization is enabled when tuning the learning rate
        if self.automatic_optimization:
            self.log('train_loss', sum(losses))
            return sum(losses)
        
        # Applying PCGrad algo
        updated_grads = []
        
        for i in range(self.n_obj):
            myopt.zero_grad()
            self.manual_backward(losses[i], retain_graph=True)

            g_task = []
            for param in self.model.parameters():
                if param.grad is not None:
                    g_task.append(Variable(param.grad.clone(), requires_grad=False))
                else:
                    g_task.append(Variable(torch.zeros(param.shape), requires_grad=False))
            # appending the gradients from each task
            updated_grads.append(g_task)

        updated_grads = list(pcgrad.pc_grad_update(updated_grads))[0]
        for idx, param in enumerate(pinn.parameters()): 
            param.grad = (updated_grads[0][idx]+updated_grads[1][idx])
            
        myopt.step()
        
        self.log('train_loss', sum(losses))
        return sum(losses)

In [18]:
ft = LightningComplexPINN(model=pinn, max_epochs=2000)
trainer = pl.Trainer(precision=32, auto_scale_batch_size=False, auto_lr_find=True, deterministic=True, amp_backend='native', max_epochs=ft.max_epochs)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [15]:
# sum(ft.model.loss(X_train[:, 0:1], X_train[:, 1:2], h_train, update_network_params=True, update_pde_params=True))

In [16]:
dataset = XYDataset(X_train, h_train)
dataloader = DataLoader(dataset, batch_size=500)

In [17]:
ft.automatic_optimization = True
trainer.tune(ft, train_dataloader=dataloader, val_dataloaders=dataloader)

/usr/local/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: you passed in a val_dataloader but have no validation_step. Skipping val loop
  warnings.warn(*args, **kwargs)

  | Name  | Type        | Params
--------------------------------------
0 | model | ComplexPINN | 81.6 K
--------------------------------------
81.6 K    Trainable params
0         Non-trainable params
81.6 K    Total params
0.326     Total estimated model params size (MB)
/usr/local/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


LR finder stopped early after 39 steps due to diverging loss.
Restored states from the checkpoint file at /Users/pongpisit/Desktop/Multi-task-Physics-informed-neural-networks/inverse_NLS/lr_find_temp_model.ckpt
Learning rate set to 7.585775750291837e-08


{'lr_find': <pytorch_lightning.tuner.lr_finder._LRFinder at 0x14b4e76d0>}

In [19]:
ft.automatic_optimization = False
trainer.fit(ft, train_dataloader=dataloader)


  | Name  | Type        | Params
--------------------------------------
0 | model | ComplexPINN | 81.6 K
--------------------------------------
81.6 K    Trainable params
0         Non-trainable params
81.6 K    Total params
0.326     Total estimated model params size (MB)
/usr/local/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


In [20]:
sum(ft.model.loss(X_train[:, 0:1], X_train[:, 1:2], h_train, update_network_params=True, update_pde_params=True))

tensor(0.0304, grad_fn=<AddBackward0>)

In [21]:
pinn = ft.model

In [23]:
epochs2 = 50
optimizer2 = torch.optim.LBFGS(pinn.parameters(), lr=1e-1, max_iter=300, max_eval=int(300*1.25), history_size=150, line_search_fn='strong_wolfe')
print('2nd Phase optimization using LBFGS')
for i in range(epochs2):
    optimizer2.step(closure)
    l = closure()
    if (i % 5) == 0 or i == epochs2-1:
        print("Epoch {}: ".format(i), l.item())

2nd Phase optimization using LBFGS
Epoch 0:  7.828387424524408e-06
Epoch 5:  7.828387424524408e-06
Epoch 10:  7.828387424524408e-06
Epoch 15:  7.828387424524408e-06
Epoch 20:  7.828387424524408e-06
Epoch 25:  7.828387424524408e-06
Epoch 30:  7.828387424524408e-06
Epoch 35:  7.828387424524408e-06
Epoch 40:  7.828387424524408e-06
Epoch 45:  7.828387424524408e-06
Epoch 49:  7.828387424524408e-06
